In [3]:
# Imports

from modules import adquire, cleaning, create_df_checking, outputs


def main(): 
    df_students, df_katas = adquire()    
    df_students, df_katas = cleaning(df_students, df_katas)
    df,hora= create_df_checking(df_students, df_katas)    
    outputs(df,False)


if __name__ == '__main__':
    main()


Shurlena presentará la kata sql-basics-monsters-using-case


slug,counting-in-the-amazon,sql-basics-monsters-using-case
username,,
E2811,True,True
eduman13,True,True
Alejorro,True,True
sowe01,True,False
meryreddoor,False,False
Cristina6692,False,False
RickAS,False,True
SantiBermejo,True,True
jmpchamizo,True,False
